In [21]:
# General purpose libraries
import os, sys
import collections
import time
from datetime import datetime, timedelta
import parsedatetime as pdt


In [22]:
# Other that I should be using
import logging
from colorama import init, Fore, Back, Style
import begin

In [23]:
#Using begin
import begin
@begin.subcommand
def main_dir_path(answer_from_terminal):
    "What is the path of main irectory where is data folder?"
    logging.info(answer_from_terminal)
@begin.subcommand
def lower_dir_path(answer_from_terminal):
    "What is the path of the lower directory where is data folder?"
    logging.info(answer_from_terminal)

ValueError: Function named 'main_dir_path' already registered

### Parsing crontab output

In [ ]:
# Change directory of output
os.chdir("/home/asisbio2/Documents/tareas/projects/semester1-2018/6_studying/01_curso_linux_efox/") # ubuntu Desktop
#os.chdir("/home/asisbio2/Documents/CENICANA/work/04_automating_server_stuff/01_curso_linux_efox/") # Ubuntu-portatil

    

In [24]:
def getting_temp_date(line):
    temp_date ="Error"
    #print(infile[idx-1])
    #print(infile[idx-1].split())
    list_date = [infile[idx].split()[0],
                infile[idx].split()[1],
                infile[idx].split()[2],
                infile[idx].split()[5],
                infile[idx].split()[3]]

    date_concatenated = " ".join(list_date)
    #print date_concatenated
    #dt, result = cal.parseDT(date_concatenated)
    #temp_date = dt.ctime()
    temp_date = date_concatenated
    
    return(temp_date)


def getting_temp_tuple(line):
    temp_name_disk_final = ""
    temp_usage=""
    temp_tuple = ("Error","Error")
    
    # Gettign list of tuples
    temp_split1 = line.split("%")
    #print(temp_split1)
    temp_name_disk = temp_split1[1].strip()

    #print(temp_split1, temp_name_disk.strip())
    if(temp_name_disk.startswith("/bio")): # Condition to get name_disk of /bio
        temp_name_disk_final = temp_name_disk
        
        # Step to get temp_usage
        len_temp = len(temp_split1[0].split("  "))
        #print(temp_split1[0].split("  "),len_temp)
        temp_usage = temp_split1[0].split("  ")[len_temp-1]
        temp_usage = temp_usage.replace(" ","")

        if(temp_usage.isdigit()):
            temp_usage = int(temp_usage)
        else:
            import re
            temp_usage_mod1 = re.split('M|K|G|T',temp_usage)
            temp_usage = temp_usage_mod1[1]
            logging.warning("Luego de split, el numero de available usage no se puede pasar a int")
            
    #temp_tuple = (temp_name_disk,tem_usage)
    
    return(temp_name_disk_final,temp_usage)

def sorting_list_tuples_for_time_series(list_sub_tuples):
    list_sub_tuples_sorted = list()
    list_ordered_disks = ["/biodata1","/biodata2", "/biodata4",
                      "/biodatatmp","/biodatatmp2", "/biodatatmp3",
                      "/bioinformatica6","/biosamba","/biotemp3"]
    
    for sortedKey in list_ordered_disks:
        for temp_tuple in list_sub_tuples:
            if(sortedKey==temp_tuple[0]):
                #print(sortedKey,temp_tuple[0],temp_tuple[1])
                temp_tuple_sorted = (temp_tuple[0],temp_tuple[1])
                list_sub_tuples_sorted.append(temp_tuple_sorted)
    return(list_sub_tuples_sorted)

# this functions is not mine
def rename1(self,key,new_key):
    ind = self._keys.index(key)  #get the index of old key, O(N) operation
    self._keys[ind] = new_key    #replace old key with new key in self._keys
    self[new_key] = self[key]    #add the new key, this is added at the end of self._keys
    self._keys.pop(-1)           #pop the last item in self._keys

def rename2(old_dict,old_name,new_name):
    new_dict = {}
    for key,value in zip(old_dict.keys(),old_dict.values()):
        new_key = key if key != old_name else new_name
        new_dict[new_key] = old_dict[key]
    return new_dict

### Running loop where the disk name and temp_usage is obtained

In [25]:
# Using collections to fill the dict
from collections import defaultdict

# Using with to not have to use f.open and f.close
infile = open("output/df_h_output_biomega_mod.txt","r").readlines()

list_aux_dates = list()
list_sub_tuples = list()
temp_tuple=("","")
dict_temp = collections.OrderedDict()
for idx,line in enumerate(infile):
   
    #print(line)
    #print(previous,line)
    if(line.startswith("lun")!=True and line.startswith("Filesystem") != True):
        temp_name_disk,temp_usage = getting_temp_tuple(line) 
        #continue
        temp_tuple = (temp_name_disk,temp_usage)
        if(not all(temp_tuple) != True):
            list_sub_tuples.append(temp_tuple)
    if(line.startswith("lun") ):
        temp_date = getting_temp_date(line)
        list_aux_dates.append(temp_date)
    
    #if(len(list_sub_tuples)==9):
    #print(temp_date,list_sub_tuples)
    
    if(line.startswith("Filesystem") == True and len(list_sub_tuples)>=9 and len(list_sub_tuples)%9==0):
            #print(idx,temp_date,len(list_sub_tuples),list_sub_tuples[(len(list_sub_tuples)-9):len(list_sub_tuples)])
            list_sub_tuples_to_sort = list_sub_tuples[(len(list_sub_tuples)-9):len(list_sub_tuples)]
            list_sub_tuples_sorted = sorting_list_tuples_for_time_series(list_sub_tuples_to_sort)
            #print(idx,temp_date,list_sub_tuples_to_sort)
            print(idx,temp_date,list_sub_tuples_sorted)
            
            dict_temp[idx] = list_sub_tuples_sorted
    
    #if(line.startswith("Filesystem") == True and len(list_sub_tuples)==0):
    #    #print(temp_date,list_sub_tuples)
    #    temp_date1 = temp_date
    #if(line.startswith("Filesystem") == True and len(list_sub_tuples)==9):    
    #        dict_temp[temp_date1]=list_sub_tuples
    #        print(temp_date1,list_sub_tuples)
    #else:
    #    if(line.startswith("Filesystem") == True and len(list_sub_tuples)>9 and len(list_sub_tuples)%9==0):
    #        print(temp_date,len(list_sub_tuples),list_sub_tuples[(len(list_sub_tuples)-9):len(list_sub_tuples)])
    
    
    

21 lun oct 15 2018 08:15:01 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 92), ('/biodatatmp', 10), ('/biodatatmp2', 10), ('/biodatatmp3', 99), ('/bioinformatica6', 71), ('/biosamba', 1), ('/biotemp3', 1)]
41 lun oct 22 2018 08:15:01 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', 10), ('/biodatatmp2', 31), ('/biodatatmp3', 99), ('/bioinformatica6', 71), ('/biosamba', 1), ('/biotemp3', 1)]
61 lun oct 29 2018 08:15:01 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', 26), ('/biodatatmp2', 34), ('/biodatatmp3', 83), ('/bioinformatica6', 71), ('/biosamba', 8), ('/biotemp3', 1)]
81 lun nov 5 2018 08:15:01 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', 26), ('/biodatatmp2', 53), ('/biodatatmp3', 83), ('/bioinformatica6', 71), ('/biosamba', 8), ('/biotemp3', 1)]
101 lun nov 12 2018 08:15:01 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', '100'), ('/biodatatmp2', 87), ('/biodatatmp3',

In [26]:
for line_reversed in reversed(infile):
    print(line_reversed.strip())

tmpfs                        303G   68K  303G   1% /run/user/1006
tmpfs                        303G  136K  303G   1% /run/user/1003
tmpfs                        303G   32K  303G   1% /run/user/108
/dev/mapper/biodisk8-vol8     10T  8,7T  786G  92% /biodatatmp3
/dev/mapper/biodisk2-vol2     10T  7,7T  1,8T  82% /biodatatmp
/dev/mapper/biodisk10-vol10   10T  9,2T  264G  98% /bioinformatica6
/dev/mapper/biodisk6-vol6     10T  9,1T  402G  96% /biodata1
/dev/mapper/biodisk9-vol9     10T  8,6T  844G  92% /biodata4
/dev/mapper/biodisk11-vol11   10T  7,4T  2,2T  78% /biotemp3
/dev/mapper/biodisk7-vol7     10T  4,0T  5,5T  43% /biodatatmp2
/dev/mapper/biodisk1-vol1     10T  8,9T  596G  94% /biodata2
/dev/mapper/biodisk4-vol4   1008G  424G  534G  45% /biosamba
tmpfs                        1,5T     0  1,5T   0% /sys/fs/cgroup
tmpfs                        5,0M     0  5,0M   0% /run/lock
tmpfs                        1,5T   90M  1,5T   1% /dev/shm
/dev/sda3                    265G  168G   84G  67% /

In [27]:
list_aux_dates[0:22]

['lun oct 8 2018 12:52:00',
 'lun oct 15 2018 08:15:01',
 'lun oct 22 2018 08:15:01',
 'lun oct 29 2018 08:15:01',
 'lun nov 5 2018 08:15:01',
 'lun nov 12 2018 08:15:01',
 'lun nov 19 2018 08:15:01',
 'lun nov 26 2018 08:15:01',
 'lun dic 3 2018 08:15:01',
 'lun dic 10 2018 08:15:01',
 'lun dic 17 2018 08:15:01',
 'lun dic 24 2018 08:15:01',
 'lun dic 31 2018 08:15:01',
 'lun ene 7 2019 08:15:01',
 'lun ene 14 2019 08:15:01',
 'lun ene 21 2019 08:15:01',
 'lun ene 28 2019 08:15:01',
 'lun feb 4 2019 08:15:01',
 'lun feb 11 2019 08:15:01',
 'lun feb 18 2019 08:15:01',
 'lun feb 25 2019 08:15:01',
 'lun mar 4 2019 08:15:02']

In [28]:
print(len(dict_temp.keys()))
print(len(list_aux_dates[0:22]))

22
22


In [29]:
# Getting new dict with keys update
dict_temp2 = collections.OrderedDict()

for new_key, old_key in zip(list_aux_dates[0:22],dict_temp.keys()):
    print(old_key,dict_temp[old_key],new_key)
    dict_temp2[new_key] = dict_temp[old_key]

21 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 92), ('/biodatatmp', 10), ('/biodatatmp2', 10), ('/biodatatmp3', 99), ('/bioinformatica6', 71), ('/biosamba', 1), ('/biotemp3', 1)] lun oct 8 2018 12:52:00
41 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', 10), ('/biodatatmp2', 31), ('/biodatatmp3', 99), ('/bioinformatica6', 71), ('/biosamba', 1), ('/biotemp3', 1)] lun oct 15 2018 08:15:01
61 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', 26), ('/biodatatmp2', 34), ('/biodatatmp3', 83), ('/bioinformatica6', 71), ('/biosamba', 8), ('/biotemp3', 1)] lun oct 22 2018 08:15:01
81 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', 26), ('/biodatatmp2', 53), ('/biodatatmp3', 83), ('/bioinformatica6', 71), ('/biosamba', 8), ('/biotemp3', 1)] lun oct 29 2018 08:15:01
101 [('/biodata1', 96), ('/biodata2', 94), ('/biodata4', 82), ('/biodatatmp', '100'), ('/biodatatmp2', 87), ('/biodatatmp3', 83), ('/bioinformatica6'

In [30]:
dict_temp2

OrderedDict([('lun oct 8 2018 12:52:00',
              [('/biodata1', 96),
               ('/biodata2', 94),
               ('/biodata4', 92),
               ('/biodatatmp', 10),
               ('/biodatatmp2', 10),
               ('/biodatatmp3', 99),
               ('/bioinformatica6', 71),
               ('/biosamba', 1),
               ('/biotemp3', 1)]),
             ('lun oct 15 2018 08:15:01',
              [('/biodata1', 96),
               ('/biodata2', 94),
               ('/biodata4', 82),
               ('/biodatatmp', 10),
               ('/biodatatmp2', 31),
               ('/biodatatmp3', 99),
               ('/bioinformatica6', 71),
               ('/biosamba', 1),
               ('/biotemp3', 1)]),
             ('lun oct 22 2018 08:15:01',
              [('/biodata1', 96),
               ('/biodata2', 94),
               ('/biodata4', 82),
               ('/biodatatmp', 26),
               ('/biodatatmp2', 34),
               ('/biodatatmp3', 83),
               ('/bio

In [31]:
list_sub_tuples_last = list()
for line_reversed in reversed(infile):
    #print(line_reversed)
    if(line_reversed.startswith("Filesystem") != True):
        temp_name_disk,temp_usage = getting_temp_tuple(line_reversed)
        temp_tuple = (temp_name_disk,temp_usage)
        
        if(not all(temp_tuple) != True):
            list_sub_tuples_last.append(temp_tuple)
            #print(temp_tuple)
    else:
        break
    #if(line.startswith("Filesystem")==True):
     #   break
list_sub_tuples_last_sorted = sorting_list_tuples_for_time_series(list_sub_tuples_last)

In [32]:
list_sub_tuples_last_sorted

[('/biodata1', 96),
 ('/biodata2', 94),
 ('/biodata4', 92),
 ('/biodatatmp', 82),
 ('/biodatatmp2', 43),
 ('/biodatatmp3', 92),
 ('/bioinformatica6', 98),
 ('/biosamba', 45),
 ('/biotemp3', 78)]

In [33]:
dict_temp2[list_aux_dates[len(list_aux_dates)-1]] = list_sub_tuples_last_sorted

In [34]:
dict_temp2

OrderedDict([('lun oct 8 2018 12:52:00',
              [('/biodata1', 96),
               ('/biodata2', 94),
               ('/biodata4', 92),
               ('/biodatatmp', 10),
               ('/biodatatmp2', 10),
               ('/biodatatmp3', 99),
               ('/bioinformatica6', 71),
               ('/biosamba', 1),
               ('/biotemp3', 1)]),
             ('lun oct 15 2018 08:15:01',
              [('/biodata1', 96),
               ('/biodata2', 94),
               ('/biodata4', 82),
               ('/biodatatmp', 10),
               ('/biodatatmp2', 31),
               ('/biodatatmp3', 99),
               ('/bioinformatica6', 71),
               ('/biosamba', 1),
               ('/biotemp3', 1)]),
             ('lun oct 22 2018 08:15:01',
              [('/biodata1', 96),
               ('/biodata2', 94),
               ('/biodata4', 82),
               ('/biodatatmp', 26),
               ('/biodatatmp2', 34),
               ('/biodatatmp3', 83),
               ('/bio

In [35]:
# Generating pandas dataframe from OrderedDict
import pandas as pd
df_final = pd.DataFrame(dict_temp2,columns=dict_temp2.keys())

In [36]:
df_final = df_final.transpose()

In [37]:
df_final

,0,1,2,3,4,5,6,7,8
lun oct 8 2018 12:52:00,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 92)","(/biodatatmp, 10)","(/biodatatmp2, 10)","(/biodatatmp3, 99)","(/bioinformatica6, 71)","(/biosamba, 1)","(/biotemp3, 1)"
lun oct 15 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 82)","(/biodatatmp, 10)","(/biodatatmp2, 31)","(/biodatatmp3, 99)","(/bioinformatica6, 71)","(/biosamba, 1)","(/biotemp3, 1)"
lun oct 22 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 82)","(/biodatatmp, 26)","(/biodatatmp2, 34)","(/biodatatmp3, 83)","(/bioinformatica6, 71)","(/biosamba, 8)","(/biotemp3, 1)"
lun oct 29 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 82)","(/biodatatmp, 26)","(/biodatatmp2, 53)","(/biodatatmp3, 83)","(/bioinformatica6, 71)","(/biosamba, 8)","(/biotemp3, 1)"
lun nov 5 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 82)","(/biodatatmp, 100)","(/biodatatmp2, 87)","(/biodatatmp3, 83)","(/bioinformatica6, 71)","(/biosamba, 8)","(/biotemp3, 1)"
lun nov 12 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 92)","(/biodatatmp, 91)","(/biodatatmp2, 91)","(/biodatatmp3, 83)","(/bioinformatica6, 95)","(/biosamba, 8)","(/biotemp3, 1)"
lun nov 19 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 92)","(/biodatatmp, 80)","(/biodatatmp2, 93)","(/biodatatmp3, 83)","(/bioinformatica6, 95)","(/biosamba, 43)","(/biotemp3, 100)"
lun nov 26 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 92)","(/biodatatmp, 80)","(/biodatatmp2, 91)","(/biodatatmp3, 83)","(/bioinformatica6, 92)","(/biosamba, 43)","(/biotemp3, 100)"
lun dic 3 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 92)","(/biodatatmp, 80)","(/biodatatmp2, 91)","(/biodatatmp3, 83)","(/bioinformatica6, 92)","(/biosamba, 44)","(/biotemp3, 100)"
lun dic 10 2018 08:15:01,"(/biodata1, 96)","(/biodata2, 94)","(/biodata4, 92)","(/biodatatmp, 80)","(/biodatatmp2, 91)","(/biodatatmp3, 83)","(/bioinformatica6, 93)","(/biosamba, 44)","(/biotemp3, 100)"


### Processing dataframe to get time series by disk_name

### Other

In [6]:
# Using collections to fill the dict
from collections import defaultdict
#tree = lambda: defaultdict(tree)
dict_by_date_of_filesytem = dict()

# creating parsedatetime object
cal = pdt.Calendar()

# Using with to not have to use f.open and f.close
infile = open("output/df_h_output_biomega.txt","r").readlines()
#previous = next(infile)

list_sub_tuples = list()
temp_tuple=("","")
for idx,line in enumerate(infile):
   
    #print(line)
    #print(previous,line)
    if(line.startswith("mar") or line.startswith("lun") ):
        temp_date = getting_temp_date(line)
    else:
        if(line.startswith("Filesystem") != True):
            temp_name_disk,temp_usage = getting_tem_tuple(line) 
            #continue
            temp_tuple = (temp_name_disk,temp_usage)
            if(not all(temp_tuple) != True):
                list_sub_tuples.append(temp_tuple)
    
    if(line.startswith("Filesystem") == True):
        print(temp_date,list_sub_tuples)
    
    #if(list_sub_tuples[len(list_sub_tuples)][0]=="/biotemp3"):
    #    print(idx,temp_date,temp_tuple)
    
    #if(not all(temp_tuple) != True):
        #print(idx,temp_date,temp_tuple)
    #print(temp_date,list_sub_tuples)
    # Filling final dict
    #dict_by_date_of_filesytem[temp_date] = list_sub_tuples
        
            

('lun oct 8 2018 12:52:00', [])
('lun oct 15 2018 08:15:01', [('/biotemp3', 1), ('/biosamba', 1), ('/biodatatmp3', 99), ('/biodatatmp', 10), ('/biodata2', 94), ('/biodata1', 96), ('/biodatatmp2', 10), ('/biodata4', 92), ('/bioinformatica6', 71)])
('lun oct 22 2018 08:15:01', [('/biotemp3', 1), ('/biosamba', 1), ('/biodatatmp3', 99), ('/biodatatmp', 10), ('/biodata2', 94), ('/biodata1', 96), ('/biodatatmp2', 10), ('/biodata4', 92), ('/bioinformatica6', 71), ('/biotemp3', 1), ('/biosamba', 1), ('/biodatatmp3', 99), ('/biodatatmp', 10), ('/biodata2', 94), ('/biodata1', 96), ('/biodatatmp2', 31), ('/biodata4', 82), ('/bioinformatica6', 71)])
('lun oct 29 2018 08:15:01', [('/biotemp3', 1), ('/biosamba', 1), ('/biodatatmp3', 99), ('/biodatatmp', 10), ('/biodata2', 94), ('/biodata1', 96), ('/biodatatmp2', 10), ('/biodata4', 92), ('/bioinformatica6', 71), ('/biotemp3', 1), ('/biosamba', 1), ('/biodatatmp3', 99), ('/biodatatmp', 10), ('/biodata2', 94), ('/biodata1', 96), ('/biodatatmp2', 31), (